<a href="https://colab.research.google.com/github/AJ-Horch/data_science_questions/blob/main/data_science_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install selenium

In [7]:
# install selenium for colab
%pip install -q google-colab-selenium

In [8]:
# get the webdriver
!pip install webdriver-manager

In [9]:
#import all libraries
import urllib.request
import requests
import bs4
from bs4 import BeautifulSoup
import matplotlib as plt
import time
import pandas as pd
import numpy as np
from selenium import webdriver
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json


In [10]:
driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

driver = gs.Chrome(options=options)

driver.get("https://towardsdatascience.com/over-100-data-scientist-interview-questions-and-answers-c5a66186769a/")

time.sleep(10)

html = driver.page_source

In [11]:
# request URLs and H2 tags
url1 = "https://towardsdatascience.com/over-100-data-scientist-interview-questions-and-answers-c5a66186769a/"
#url2 = "https://towardsdatascience.com/25-a-b-testing-concepts-interview-cheat-sheet-c998a501f911/"
#url3 = "https://www.datacamp.com/blog/data-scientist-interview-questions"
#url4 = "https://www.datacamp.com/blog/top-machine-learning-interview-questions"
#url5 = "https://towardsdatascience.com/probability-interview-questions-in-real-life-of-data-analysts-0c8b5953abd8/"


In [12]:
driver.get(url1)
print(driver.title)
driver.quit()

OVER 100 Data Scientist Interview Questions and Answers! | Towards Data Science


All of the questions are tagged with h3 - aka subheaders. Typically in online publushing there is a headline, a header and then a subheader. In the case of this article url1, the subheader is the question.

In [13]:

# Setup headless Chrome browser
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Initialize browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Request URL
url = url1
driver.get(url)

# Wait for content to load
time.sleep(5)

# Get page source after JavaScript execution
html = driver.page_source
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(html, 'lxml')

# Print h3 tags
h3_tags = soup.find_all('h3')
print(f"Found {len(h3_tags)} h3 tags")
print("First four h3 tags from the scraped webpage:")
for i, tag in enumerate(h3_tags[:4]):
    print(f"{i+1}. {tag.text}")

Found 99 h3 tags
First four h3 tags from the scraped webpage:
1. Q: What are some of the steps for data wrangling and data cleaning before applying machine learning algorithms?
2. Q: How to deal with unbalanced binary classification?
3. Q: What is the difference between a box plot and a histogram?
4. Q: Describe different regularization methods, such as L1 and L2 regularization?


## Returning the questions
We've found 99 h3 tags - aka subheaders. The unique identifier for each h3 questions is its id tag. The following code creates a dictionary of key, value pairs where the h2 tag id is the key and h3 text, the actual questions,is the value.

In [14]:
## Find and add all H3 tag IDs to a list
tag_list = list()
question_dict = dict()

for tag in soup.find_all('h3')[:-1]:
    if tag.text[:2] == 'Q:' and tag.get('id') != None:
        tag_list.append(tag.get('id'))
        question_dict[tag.get('id')] = tag.text

for tag_id in tag_list[:4]:
    print(question_dict[tag_id])

In [15]:
def extract_questions_and_answers():
    # Setup headless Chrome browser
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    # Initialize browser
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Request URL
    url = 'https://towardsdatascience.com/over-100-data-scientist-interview-questions-and-answers-c5a66186769a'
    print(f"Fetching content from: {url}")
    driver.get(url)

    # Wait for content to load
    print("Waiting for page to load completely...")
    time.sleep(8)  # Increasing wait time to ensure content loads

    # Get page source after JavaScript execution
    html = driver.page_source
    driver.quit()

    # Parse with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find all h2 sections (categories)
    categories = soup.find_all('h2', class_='wp-block-heading')

    # Lists to store data for DataFrame
    category_list = []
    question_list = []
    answer_list = []

    # Process each category
    for category in categories:
        current_category = category.text.strip()
        print(f"\nProcessing category: {current_category}")

        # Find all elements after this h2 until the next h2
        next_elements = []
        for sibling in category.find_next_siblings():
            if sibling.name == 'h2':
                break
            next_elements.append(sibling)

        # Find questions and answers within this section
        for i in range(len(next_elements)):
            element = next_elements[i]

            # If we find a question (h3 tag)
            if element.name == 'h3' and element.get('class') and 'wp-block-heading' in element.get('class'):
                question_text = element.text.strip()

                # Look for the answer (p tag) after the question
                answer_texts = []
                j = i + 1

                # Collect all p tags until we hit another h3 or h2
                while j < len(next_elements):
                    next_element = next_elements[j]
                    if next_element.name == 'h3' or next_element.name == 'h2':
                        break

                    # Add paragraphs to answer
                    if next_element.name == 'p' and next_element.get('class') and 'wp-block-paragraph' in next_element.get('class'):
                        answer_texts.append(next_element.text.strip())

                    # Also capture list items if they exist
                    if next_element.name == 'ul' or next_element.name == 'ol':
                        list_items = next_element.find_all('li')
                        list_text = "\n".join([f"• {li.text.strip()}" for li in list_items])
                        if list_text:
                            answer_texts.append(list_text)

                    j += 1

                # Combine all answer parts
                answer = "\n\n".join(answer_texts)

                # Append to lists for DataFrame
                category_list.append(current_category)
                question_list.append(question_text)
                answer_list.append(answer)

                print(f"  Extracted: {question_text[:50]}...")

    # Create DataFrame
    df = pd.DataFrame({
        'Category': category_list,
        'Question': question_list,
        'Answer': answer_list
    })

    print(f"\nTotal QA pairs extracted: {len(df)}")
    return df

def main():
    # Extract data and create DataFrame
    qa_df = extract_questions_and_answers()

    # Display DataFrame info
    print("\n=== DATAFRAME INFO ===")
    print(qa_df.info())

    # Display a few examples
    print("\n=== SAMPLE ROWS ===")
    print(qa_df.head(2))

    # Save to CSV file
    csv_path = 'medium_qa_pairs.csv'
    qa_df.to_csv(csv_path, index=False, encoding='utf-8')
    print(f"\nDataFrame saved to {csv_path}")

    # Save to Excel file for better formatting
    excel_path = 'medium_qa_pairs.xlsx'
    qa_df.to_excel(excel_path, index=False)
    print(f"DataFrame also saved to {excel_path}")

    # Save to pickled DataFrame for Python use
    pickle_path = 'medium_qa_pairs.pkl'
    qa_df.to_pickle(pickle_path)
    print(f"DataFrame also saved as pickle to {pickle_path}")

if __name__ == "__main__":
    main()


Fetching content from: https://towardsdatascience.com/over-100-data-scientist-interview-questions-and-answers-c5a66186769a
Waiting for page to load completely...

Processing category: Table of Content

Processing category: Machine Learning Fundamentals
  Extracted: Q: What are some of the steps for data wrangling a...
  Extracted: Q: How to deal with unbalanced binary classificati...
  Extracted: Q: What is the difference between a box plot and a...
  Extracted: Q: Describe different regularization methods, such...
  Extracted: Q: Neural Network Fundamentals...
  Extracted: Q: What is cross-validation?...
  Extracted: Q: How to define/select metrics?...
  Extracted: Q: Explain what precision and recall are...
  Extracted: Q: Explain what a false positive and a false negat...
  Extracted: Q: What is the difference between supervised learn...
  Extracted: Q: Assume you need to generate a predictive model ...
  Extracted: Q: What does NLP stand for?...
  Extracted: Q: When would you use r

In [16]:
qadf = pd.read_csv('medium_qa_pairs.csv')
qadf.head()

,Category,Question,Answer
0,Machine Learning Fundamentals,Q: What are some of the steps for data wrangli...,There are many steps that can be taken when da...
1,Machine Learning Fundamentals,Q: How to deal with unbalanced binary classifi...,There are a number of ways to handle unbalance...
2,Machine Learning Fundamentals,Q: What is the difference between a box plot a...,While boxplots and histograms are visualizatio...
3,Machine Learning Fundamentals,"Q: Describe different regularization methods, ...",Both L1 and L2 regularization are methods used...
4,Machine Learning Fundamentals,Q: Neural Network Fundamentals,A neural network is a multi-layered model insp...


In [19]:
qadf.sample(4)

,Category,Question,Answer
57,"Statistics, Probability, and Mathematics","Q: When you sample, what bias are you inflicting?",Potential biases include the following:\n\n• S...
10,Machine Learning Fundamentals,Q: Assume you need to generate a predictive mo...,There are two main ways that you can do this:\...
17,Machine Learning Fundamentals,Q: Do you think 50 small decision trees are be...,"Another way of asking this question is ""Is a r..."
33,"Statistics, Probability, and Mathematics",Q: A box has 12 red cards and 12 black cards. ...,The box with 24 red cards and 24 black cards h...


## Creating emial
The following python script will read the newly created Q&A DF. Then it will randomly selected a pair and send an emial to my gmail account.


In [56]:
# import libraies
import smtplib
import random
import io
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google.colab import files
import datetime
import re
import html
import smtplib
import ssl

In [50]:
# A function to select a random QA pair
def get_random_question():
  # select random row
  random_row = qadf.sample(1).iloc[0]

  #extract the category, question and answer
  category = random_row['Category']
  question = random_row["Question"]
  answer = random_row["Answer"]

  return category, question, answer


Script is ready! Update the email credentials and uncomment the send_daily_question line to run it.


In [29]:
# Test the function multiple times
print("Testing get_random_question() function:")
for i in range(3):
    category, question, answer = get_random_question()
    print(f"\nTest {i+1}:")
    print(f"Category: {category}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")

Testing get_random_question() function:

Test 1:
Category: Statistics, Probability, and Mathematics
Question: Q: What is the Law of Large Numbers?
Answer: The Law of Large Numbers is a theory that states that as the number of trials increases, the average of the result will become closer to the expected value.

Eg. flipping heads from fair coin 100,000 times should be closer to 0.5 than 100 times.

Test 2:
Category: Statistics, Probability, and Mathematics
Question: Q: Is mean imputation of missing data acceptable practice? Why or why not?
Answer: Mean imputation is the practice of replacing null values in a data set with the mean of the data.

Mean imputation is generally bad practice because it doesn’t take into account feature correlation. For example, imagine we have a table showing age and fitness score and imagine that an eighty-year-old has a missing fitness score. If we took the average fitness score from an age range of 15 to 80, then the eighty-year-old will appear to have a 

In [53]:
category, question, answer = get_random_question()
print(f"Category: {category}\nQuestion: {question}\nAnswer: {answer}")

Category: Machine Learning Fundamentals
Question: Q: Neural Network Fundamentals
Answer: A neural network is a multi-layered model inspired by the human brain. Like the neurons in our brain, the circles above represent a node. The blue circles represent the input layer, the black circles represent the hidden layers, and the green circles represent the output layer. Each node in the hidden layers represents a function that the inputs go through, ultimately leading to an output in the green circles. The formal term for these functions is called the sigmoid activation function.

If you want a step by step example of creating a neural network, check out Victor Zhou’s article here.

If you’re a visual/audio learner, 3Blue1Brown has an amazing series on neural networks and deep learning on YouTube here.


In [35]:
question

'Q: Likes/user and minutes spent on a platform are increasing but total number of users are decreasing. What could be the root cause of it?'

In [38]:
get_random_question()

('Statistics, Probability, and Mathematics',
 'Q: You roll a biased coin (p(head)=0.8) five times. What’s the probability of getting three or more heads?',
 'Use the General Binomial Probability formula to answer this question:\n\np = 0.8\nn = 5\nk = 3,4,5\n\nP(3 or more heads) = P(3 heads) + P(4 heads) + P(5 heads) = 0.94 or 94%')

In [54]:
def get_random_question():
    random_row = qadf.sample(1).iloc[0]
    category = random_row['Category']
    question = random_row['Question']
    answer = random_row['Answer']
    return category, question, answer

def send_email(sender_email, sender_password, recipient_email):
    category, question, answer = get_random_question()

    subject = f"Daily QA - {category}"
    body = f"""
    <h2>📚 {category}</h2>
    <p><strong>Question:</strong> {question}</p>
    <p><strong>Answer:</strong> {answer}</p>
    """

    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = recipient_email
    part = MIMEText(body, "html")
    msg.attach(part)

    # Email sending
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())

In [61]:
# Send a random email.
send_email(
    sender_email="josephhorch@gmail.com",
    sender_password="nlou jctu hxok axzo",
    recipient_email="josephhorch@gmail.com"
)